CONTENTS LIST:
- Learning user-defined functions
- Using .loc function to categorize data
- Using for-loops

#01 Import libraries & data

In [3]:
import pandas as pd
import numpy as np
import os

In [4]:
path = '/Users/gingermoore/Documents/04-2025 Instacart Basket Analysis'

In [5]:
df_ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge.pkl'))

In [6]:
df_ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,orders_chronological,orders_day_of_week,order_hour_of_day,days_since_prior_order,is_new,add_to_cart_order,reordered,exists
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,False,5,0,both
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,False,1,1,both
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,False,20,0,both
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,NaN,True,10,0,both
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,False,11,1,both


In [7]:
df = df_ords_prods_merge[:1000000]

User-defined functions

In [9]:
def price_label(row):

  if row['prices'] <= 5:
    return 'Low-range product'
  elif (row['prices'] > 5) and (row['prices'] <= 15):
    return 'Mid-range product'
  elif row['prices'] > 15:
    return 'High range'
  else: return 'Not enough data'

In [10]:
df['price_range'] = df.apply(price_label, axis=1)

/var/folders/dt/_2cmd91j32bgtvjmx8m320w40000gn/T/ipykernel_63709/2848743372.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_range'] = df.apply(price_label, axis=1)


I did some research after encountering the error above and found that the warning can be disabled using the following code.

In [12]:
pd.options.mode.chained_assignment = None  # default='warn'

In [13]:
df['price_range'] = df.apply(price_label, axis=1)

In [14]:
df['price_range'].value_counts(dropna = False)

price_range
Mid-range product    652638
Low-range product    338018
High range             9344
Name: count, dtype: int64

In [15]:
df['prices'].max()

24.5

In [16]:
df.shape

(1000000, 16)

Using the .loc() function

In [18]:
df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [19]:
df.loc[(df['prices'] <= 15) & (df['prices'] > 5), 'price_range_loc'] = 'Mid-range product' 

In [20]:
df.loc[df['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [21]:
df['price_range_loc'].value_counts(dropna = False)

price_range_loc
Mid-range product     652638
Low-range product     338018
High-range product      9344
Name: count, dtype: int64

For-loops

In [23]:
result = []

for value in df["orders_day_of_week"]:
  if value == 0:
    result.append("Busiest day")
  elif value == 4:
    result.append("Least busy")
  else:
    result.append("Regularly busy")

In [25]:
df['busiest_day'] = result

In [26]:
df['busiest_day'].value_counts(dropna = False)

busiest_day
Regularly busy    696112
Busiest day       183956
Least busy        119932
Name: count, dtype: int64

#02 Working on the task

If we want to know the two busiest days, we should be able to simpy modify the 'result' value. So we'd do something like this:

In [29]:
result = []
for value in df["orders_day_of_week"]:
  if value == 0 or 1:
    result.append("Busiest day")
  elif value == 3 or 4:
    result.append("Least busy")
  else:
    result.append("Regularly busy")

In [30]:
#Creating new columns per request
df['busiest_days'] = result

In [31]:
df['busiest_days'].value_counts(dropna = False)

busiest_days
Busiest day    1000000
Name: count, dtype: int64

Okay well that did not work. Let me try something else:

In [33]:
result = []
for value in df["orders_day_of_week"]:
  if value == [0,1]:
    result.append("Busiest day")
  elif value == [3,4]:
    result.append("Least busy")
  else:
    result.append("Regularly busy")

In [34]:
df['busiest_days'] = result

In [35]:
df['busiest_days'].value_counts(dropna = False)

busiest_days
Regularly busy    1000000
Name: count, dtype: int64

That didn't work either. Maybe I just need to add additional lines of code, like so?

In [37]:
result = []
for value in df["orders_day_of_week"]:
  if value == 0:
    result.append("Busiest day")
  elif value == 1:
    result.append("Busiest day")
  elif value == 3:
    result.append("Least busy")
  elif value == 4:
    result.append("Least busy")
  else:
    result.append("Regularly busy")

In [38]:
df['busiest_days'] = result

In [39]:
df['busiest_days'].value_counts(dropna = False)

busiest_days
Regularly busy    401159
Busiest day       358026
Least busy        240815
Name: count, dtype: int64

I got it! I guess I did have to add additional lines of code. Is there not a more efficient way to do this? I feel like there should be...

Update: I used ChatGPT to debug and figured out the answer, will use it in the hour of day for loop

In [41]:
#Printing head because I don't remember the column names lol
df.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,orders_chronological,orders_day_of_week,order_hour_of_day,days_since_prior_order,is_new,add_to_cart_order,reordered,exists,price_range,price_range_loc,busiest_day,busiest_days
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,False,5,0,both,Mid-range product,Mid-range product,Regularly busy,Regularly busy
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,False,1,1,both,Mid-range product,Mid-range product,Regularly busy,Regularly busy
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,False,20,0,both,Mid-range product,Mid-range product,Busiest day,Busiest day
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,NaN,True,10,0,both,Mid-range product,Mid-range product,Regularly busy,Least busy
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,False,11,1,both,Mid-range product,Mid-range product,Least busy,Least busy


In [42]:
#Finding frequency of order_hour_of_day
df['order_hour_of_day'].value_counts(dropna = False)

order_hour_of_day
10    86241
11    84780
14    83070
13    82272
15    81234
12    80541
16    77784
9     76870
17    64196
8     53294
18    50082
19    38584
20    30367
7     27542
21    24338
22    19126
23    12156
6      9090
0      6560
1      3576
5      2831
2      2174
3      1653
4      1639
Name: count, dtype: int64

I've been asked to divide the hours by most, average, and fewest amount of orders, so I'm going to put 8 hours in each category, like so:

In [44]:
hour_range = []
for value in df['order_hour_of_day']:
    if value in [9,10,11,12,13,14,15,16]:
        hour_range.append('Most orders')
    elif value in [7,8,17,18,19,20,21,22]:
        hour_range.append('Average orders')
    else: hour_range.append('Fewest orders')

In [45]:
df['busiest_period_of_day'] = hour_range

In [46]:
df['busiest_period_of_day'].value_counts(dropna = False)

busiest_period_of_day
Most orders       652792
Average orders    307529
Fewest orders      39679
Name: count, dtype: int64

In [48]:
df_ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,orders_chronological,orders_day_of_week,order_hour_of_day,days_since_prior_order,is_new,add_to_cart_order,reordered,exists
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,False,5,0,both
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,False,1,1,both
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,False,20,0,both
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,NaN,True,10,0,both
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,False,11,1,both


Making adjustments as I did not correctly export what I meant to. Need to rerun some code.

In [51]:
df_ords_prods_merge['price_range'] = df_ords_prods_merge.apply(price_label, axis=1)

In [52]:
df_ords_prods_merge['price_range'].value_counts(dropna = False)

price_range
Mid-range product    21860860
Low-range product    10126321
High range             412551
Name: count, dtype: int64

In [55]:
df_ords_prods_merge.loc[df_ords_prods_merge['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [57]:
df_ords_prods_merge.loc[(df_ords_prods_merge['prices'] <= 15) & (df_ords_prods_merge['prices'] > 5), 'price_range_loc'] = 'Mid-range product' 

In [59]:
df_ords_prods_merge.loc[df_ords_prods_merge['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [61]:
df_ords_prods_merge['price_range_loc'].value_counts(dropna = False)

price_range_loc
Mid-range product     21860860
Low-range product     10126321
High-range product      412551
Name: count, dtype: int64

In [65]:
result = []
for value in df_ords_prods_merge["orders_day_of_week"]:
  if value in [0,1]:
    result.append("Busiest days")
  elif value in [3,4]:
    result.append("Least busy days")
  else:
    result.append("Regularly busy")

In [67]:
df_ords_prods_merge['busiest_days'] = result

In [69]:
df_ords_prods_merge['busiest_days'].value_counts(dropna = False)

busiest_days
Regularly busy     12914068
Busiest days       11862627
Least busy days     7623037
Name: count, dtype: int64

In [71]:
df_ords_prods_merge['orders_day_of_week'].value_counts(dropna = False)

orders_day_of_week
0    6203329
1    5659298
6    4495887
2    4213105
5    4205076
3    3839865
4    3783172
Name: count, dtype: int64

In [73]:
df_ords_prods_merge['order_hour_of_day'].value_counts(dropna = False)

order_hour_of_day
10    2761333
11    2735694
14    2688728
15    2661718
13    2660570
12    2618104
16    2534744
9     2453842
17    2087273
8     1717863
18    1636226
19    1258076
20     976000
7      890923
21     795528
22     634159
23     402272
6      290450
0      218742
1      115683
5       87944
2       69360
4       53232
3       51268
Name: count, dtype: int64

In [75]:
hour_range = []
for value in df_ords_prods_merge['order_hour_of_day']:
    if value in [9,10,11,12,13,14,15,16]:
        hour_range.append('Most orders')
    elif value in [7,8,17,18,19,20,21,22]:
        hour_range.append('Average orders')
    else: hour_range.append('Fewest orders')

In [77]:
df_ords_prods_merge['busiest_period_of_day'] = hour_range

In [79]:
df_ords_prods_merge['busiest_period_of_day'].value_counts(dropna = False)

busiest_period_of_day
Most orders       21114733
Average orders     9996048
Fewest orders      1288951
Name: count, dtype: int64

In [81]:
df_ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,orders_chronological,orders_day_of_week,order_hour_of_day,days_since_prior_order,is_new,add_to_cart_order,reordered,exists,price_range,price_range_loc,busiest_days,busiest_period_of_day
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,False,5,0,both,Mid-range product,Mid-range product,Regularly busy,Most orders
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,False,1,1,both,Mid-range product,Mid-range product,Regularly busy,Average orders
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,False,20,0,both,Mid-range product,Mid-range product,Busiest days,Average orders
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,NaN,True,10,0,both,Mid-range product,Mid-range product,Least busy days,Most orders
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,False,11,1,both,Mid-range product,Mid-range product,Least busy days,Average orders


In [83]:
df_opm_newcols = df_ords_prods_merge.drop(columns = 'price_range_loc')

In [85]:
df_opm_newcols.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,orders_chronological,orders_day_of_week,order_hour_of_day,days_since_prior_order,is_new,add_to_cart_order,reordered,exists,price_range,busiest_days,busiest_period_of_day
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,False,5,0,both,Mid-range product,Regularly busy,Most orders
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,False,1,1,both,Mid-range product,Regularly busy,Average orders
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,False,20,0,both,Mid-range product,Busiest days,Average orders
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,NaN,True,10,0,both,Mid-range product,Least busy days,Most orders
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,False,11,1,both,Mid-range product,Least busy days,Average orders


In [87]:
df_opm_newcols.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'new_ords_prods_merge.pkl'))